*This is some code for solving the computational exercises from Week 1.* $\DeclareMathOperator{\LT}{LT}$

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.13.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


<div class="alert alert-block alert-info">
<p>We previously recommended the Oscar function <code>divrem</code> for multivariate divison, but this has the unusual behavior that  the user cannot supply a monomial order as input. Instead, the monomial order needs to be defined <em>already when the ring is defined</em>. Since this is very inconvenient and easilly causes confusion, we recommend that you don't use it, and instead use the following "homemade" function <code>multivariate_division</code>.</p>

<p><strong>Note:</strong> You don't need to understand all the commands and Julia syntax we're using in this function, but it's a good exercise to compare the overall structure to the pseudocode we give on the exercise sheet!</p>
</div>

In [2]:
function multivariate_division(f::MPolyRingElem, F::Vector{<:MPolyRingElem}; ordering::MonomialOrdering)
    LT = (h -> leading_term(h,ordering=ordering))
    s = length(F)
    Q = [zero(parent(f)) for i=1:s]
    r = 0
    p = f
    while p != 0
        division_occured = false
        for i = 1:s
            division_occured, quotient = divides( LT(p), LT(F[i]) )
            if division_occured 
                Q[i] = Q[i] + quotient
                p = p - quotient*F[i]
                break
            end
        end
        if !(division_occured)
            r = r + LT(p)
            p = p - LT(p)
        end
    end
    return Q,r
end;

Note that for this function you have to specify with respect to which **monomial ordering** you want to work.

Here are the commands for some of the most common monomial orderings one can work with in a polynomial ring $R=k[x_1,\ldots,x_n]$ in Oscar:

- `lex(R)` (lexiographic ordering, $<_\text{lex}$)
- `deglex(R)`(graded lexiographic ordering, $<_\text{grlex}$)
- `degrevlex(R)`(graded lexiographic ordering, $<_\text{grevlex}$)
- `matrix_ordering(R,M)` (the matrix ordering $<_M$ from Exercise 8, for some real-valued matrix $M$ with $n$ columns)

## Exercise 19(a)

In [3]:
R, (x,y) = polynomial_ring(QQ, ["x","y"]);

In [4]:
f = x^2 + x*y^2 + y;
f1 = x + x*y;
f2 = x+y;

We begin by diving $f$ by $(f_1,f_2)$, and get the following quotient $Q$ and remainder $r$:

In [5]:
Q,r = multivariate_division(f, [f1,f2], ordering=lex(R));

In [6]:
Q

2-element Vector{QQMPolyRingElem}:
 y - 2
 x + 2

In [7]:
r

-y

Next, we divide $f$ by $(f_2,f_1)$ instead, and get the following quotient and remainder:

In [8]:
Q, r = multivariate_division(f, [f2,f1], ordering=lex(R));

In [9]:
Q

2-element Vector{QQMPolyRingElem}:
 x + y^2 - y
 0

In [10]:
r

-y^3 + y^2 + y

## Exercise 19(b)

We start by making the division from Exercise 18 with the **lex order** $x>y$ (called `lex` in Oscar):

In [11]:
R, (x,y) = polynomial_ring(QQ, ["x","y"]);
f = x^2*y + x*y^2 + y^2;
f1 = x*y - 1;
f2 = y^2 - 1;
Q, r = multivariate_division(f, [f1,f2], ordering=lex(R));

In [12]:
Q

2-element Vector{QQMPolyRingElem}:
 x + y
 1

In [13]:
r

x + y + 1

Next, we try the division with the **grlex order** $x>y$ (called `deglex` in Oscar).

In [14]:
Q, r = multivariate_division(f,[f1,f2],ordering=deglex(R));

In [15]:
Q

2-element Vector{QQMPolyRingElem}:
 x + y
 1

In [16]:
r

x + y + 1

## Exercise 20

**(a)** We divide $f$ by $(f_1,f_2)$ and $(f_2,f_1)$, as follows:

In [17]:
R, (x,y,z) = polynomial_ring(QQ, ["x","y","z"]);
f = x^3 - x^2*y - x^2*z + x;
f1 = x^2*y - z;
f2 = x*y - 1;

In [18]:
Q, r = multivariate_division(f,[f1,f2],ordering=deglex(R))

(QQMPolyRingElem[-1, 0], x^3 - x^2*z + x - z)

In [19]:
Qprime, rprime = multivariate_division(f,[f2,f1],ordering=deglex(R))

(QQMPolyRingElem[-x, 0], x^3 - x^2*z)

**(b)** We have that 
$$f=(q_1,q_2)\bullet (f_1,f_2)+r$$ $$f=(q_1',q_2')\bullet (f_2,f_1)+r'=(q_2',q_1')\bullet (f_1,f_2)+r'.$$
Hence, 
$$g:=r-r'=(q_2',q_1')\bullet (f_1,f_2)-(q_1,q_2)\bullet (f_1,f_2)=(q_2'-q_1,q_1'-q_2)\bullet (f_1,f_2)=\underbrace{(q_2'-q_1)}_A f_1+\underbrace{(q_1'-q_2)}_B f_2$$

In [20]:
g = r - rprime

x - z

In [21]:
A = Qprime[2] - Q[1]

1

In [22]:
B = Qprime[1] - Q[2]

-x

In [23]:
A*f1 + B*f2

x - z

**(c)** We just get back $g$ as the remainder, since none of the monomials appearing in $g$ are divisible by any of the monomials $\LT(f_1)=x^2y$ and $\LT(f_1)=xy$.

In [24]:
Qg, rg = multivariate_division(g,[f1,f2],ordering=deglex(R))

(QQMPolyRingElem[0, 0], x - z)

**(d)** The trick is to cancel the leading terms of $f_1$ and $f_2$ in some way, such that we get a $\LT(h)$ that is not divisible by $xy$. Another way (in fact: the only fundamentally different way) to do this is as follows:

In [25]:
h = y*f1 - (x*y+1)*f2

-y*z + 1

In [26]:
multivariate_division(h,[f1,f2],ordering=deglex(R))

(QQMPolyRingElem[0, 0], -y*z + 1)

**(e)** Part (c) and (d) shows that the division algorithm on its own *doesn't* solve the ideal membership problem.

**(f)** Let $I=\langle f_1,f_2\rangle$. If we want to test ideal membership with division, we at the very least need to find another basis $I=\langle g_1,\ldots,g_r\rangle$ of the ideal, such that for any $h\in I$, it holds that $\operatorname{LT}(h)$ is divisible by one of the monomials $\LT(g_1),\ldots,\LT(g_r)$ -- if not, we inevitably will get a remainder! 

Next week, we will see that this this corresponds to finding a *Gröbner basis* of $I$, and that this is not only necessary but also sufficient to solve the ideal membership problem. For this ideal, we will see that the generating set $\{x-z,xy-1\}$ constitutes a Gröbner basis.

## Exercise 22

**(a)** We first try to do the division in $\mathbb{Q}[x,y]$ with the **lexiographic ordering** with $x>y$. 

In [27]:
R, (x,y) = polynomial_ring(QQ, ["x","y"]);
f = x^7*y^2 + x^3*y^2 - y + 1;
f1 = x*y^2 - x;
f2 = x - y^3;

First we divide $f$ by $(f_1,f_2)$:

In [28]:
Q, r = multivariate_division(f,[f1,f2],ordering=lex(R));

In [29]:
Q

2-element Vector{QQMPolyRingElem}:
 x^6 + x^5*y + x^4*y^2 + x^4 + x^3*y + x^2*y^2 + 2*x^2 + 2*x*y + 2*y^2 + 2
 x^6 + x^5*y + x^4 + x^3*y + 2*x^2 + 2*x*y + 2

In [30]:
r

2*y^3 - y + 1

Then we divide $f$ by $(f_2,f_1)$ instead.

In [31]:
Q, r = multivariate_division(f,[f2,f1],ordering=lex(R));

In [32]:
Q

2-element Vector{QQMPolyRingElem}:
 x^6*y^2 + x^5*y^5 + x^4*y^8 + x^3*y^11 + x^2*y^14 + x^2*y^2 + x*y^17 + x*y^5 + y^20 + y^8
 0

In [33]:
r

y^23 + y^11 - y + 1

**(b)** Now we do the same with respect to the **graded lexiographic ordering** with $x>y$. 

In [34]:
Q, r = multivariate_division(f,[f1,f2],ordering=deglex(R));

In [35]:
Q

2-element Vector{QQMPolyRingElem}:
 x^6 + x^2
 0

In [36]:
r

x^7 + x^3 - y + 1

In [37]:
Q, r = multivariate_division(f,[f2,f1],ordering=deglex(R));

In [38]:
Q

2-element Vector{QQMPolyRingElem}:
 0
 x^6 + x^2

In [39]:
r

x^7 + x^3 - y + 1